# Working with databases

We halen alle films met nominaties & awards uit wikipedia (https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films) en bewaren ze in een databank

Installeer package mysql-connector-python https://dev.mysql.com/doc/connector-python/en/

In [ ]:
! pip install mysql-connector-python

## Connectie met databank in Python

Gebruik Xampp (of Mamp) om een mysql-database 'oscars' op te zetten met een tabel 'movies'

<pre>
CREATE TABLE `movies` ( 
  `id` int(11) NOT NULL, 
  `title` varchar(100) NOT NULL, 
  `year` year(4) NOT NULL, 
  `awards` tinyint(4) NOT NULL, 
  `nominations` tinyint(4) NOT NULL 
) 
</pre>

Standaard gebruikersnaam & paswoord van phpmyadmin (Xampp) zijn 'root' en '' 

In [9]:
import mysql.connector

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="oscars"
)

cursor = db.cursor()

## Wikipedia scrapen en wegschrijven naar tabel 'movies'

In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films'
r = requests.get(url)
soup = BeautifulSoup(r.text) 
table = soup.find('table',{ "class" : "wikitable sortable" }) # tabel zoeken
count=0

for tr in table.find_all('tr'):     # per elke tr --> row dus in tabel
    row = tr.find_all('td')         # zoeken we naar kolommen td 
    if (len(row)==4):               # als er 4 kolommen zijn, zijn we normaal gezien zeker dat het de juiste info is
        title = row[0].text.strip() 
        year = row[1].text.strip()
        awards = row[2].text.strip()
        nominations = row[3].text.strip()

        # probeer rij toe te voegen aan tabel
        try:
            sql = f"INSERT INTO movies (title, year, awards, nominations) VALUES (\"{title}\",{year},{awards},{nominations})"
            cursor.execute(sql)
            db.commit()
            count+=1
        except: #lukt het niet, negeer het dan gewoon ;)
            pass

print(count, "rows added")


## Tabel bevragen

In [18]:
# zoek de 5 films met hoogst aantal awards
cursor.execute("SELECT * FROM movies ORDER BY awards DESC limit 5")
results = cursor.fetchall()

for movie in results:
  print(movie)

(817, 'Ben-Hur', 1959, 11, 12)
(352, 'Titanic', 1997, 11, 14)
(270, 'The Lord of the Rings: The Return of the King', 2003, 11, 11)
(1256, 'West Side Story', 1961, 10, 11)
(938, 'Gigi', 1958, 9, 9)
